In [13]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   On_Black="\033[40m"
   On_Red="\033[41m"
   On_Green="\033[42m"
   On_Yellow="\033[43m"
   On_Blue="\033[44m"
   On_Purple="\033[45m"
   On_Cyan="\033[46m"
   On_White="\033[47m"
   On_IBlue="\033[0;104m"
   END = '\033[0m'

In [1]:
import random
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Load CIFAR-10 dataset

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
input_shape = x_train.shape[1:]

170498071/170498071 [==============================] - 72s 0us/step


# Define the search space for NAS

In [3]:
search_space = {
    'num_conv_layers': [ConV for ConV in range(1, 7)],
    'num_filters': [num_filer_ for num_filer_ in range(2, 16)],
    'num_dense_units': [num_dense_units_ for num_dense_units_ in range(128, 1024)],
    'kernel_size': [kernel_size_ for kernel_size_ in range(2, 7)],
    'pool_size': [pool_size_ for pool_size_ in range(2, 5)],
    #'learning_rate': [0.0001, 0.001, 0.01, 0.1]
}

# Generate an initial population of neural architectures

In [4]:
def generate_population(size):
    population = []
    print(color.On_Purple+"Initial population of neural architectures--------------------------------------------------------------------------"+color.END)
    Count_for_pop = 1
    for _ in range(size):
        architecture = {}
        for param, choices in search_space.items():
            architecture[param] = random.choice(choices)
        population.append(architecture)
        print("Population:", Count_for_pop, architecture)
        Count_for_pop += 1
    print(color.On_Purple+"--------------------------------------------------------------------------------------------------------------------"+color.END)
    return population

# Evaluate the architectures in the population

In [15]:
epochs = 5
batch_size = 128
def evaluate_architecture(architecture, X_val=x_val, Y_val=y_val ):
    model = Sequential()
    # Add convolutional layers
    for _ in range(int(architecture['num_conv_layers'])):
        model.add(Conv2D(int(architecture['num_filters']), (int(architecture['kernel_size']),
                                                             int(architecture['kernel_size'])),
                         activation='relu', padding='same', input_shape=input_shape))
        model.add(MaxPooling2D(pool_size=(int(architecture['pool_size']), int(architecture['pool_size'])), padding='same'))

    model.add(Flatten())

    # Add dense layers
    model.add(Dense(architecture['num_dense_units'], activation='relu'))

    # Output layer
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    optimizer = Adam()
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    # Train the model
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0)

    # Evaluate the model on the validation set
    _, accuracy = model.evaluate(X_val, Y_val, verbose=0)
    return accuracy

# Select parents

In [6]:
def select_parents(population, num_parents):
    print(color.GREEN+"Select parents for reproduction"+color.END)
    parents = random.sample(population, num_parents)
    for item in parents:
        print(item)
    return parents

# Select parents for reproduction using *******roulette wheel selection*********
#def select_parents(population, num_parents):
    #print("Select parents for reproduction")
    #fitness_values = [individual['fitness'] for individual in population]
    #sum_fitness = sum(fitness_values)
    #probabilities = [fitness / sum_fitness for fitness in fitness_values]
    #parents = random.choices(population, probabilities, k=num_parents)
    #for item in parents:
        #print(item)
    #return parents

# Generate offspring architectures using genetic operators

In [7]:
def generate_offspring(parents, num_offspring, mutation_rate):
    print(color.GREEN+"Generate offspring architectures using genetic operators:"+color.END)
    offspring = []
    OFF_Count = 1
    for _ in range(num_offspring):
        parent1, parent2 = random.choices(parents, k=2)
        while parent1 == parent2:  # Repeat selection if parent1 and parent2 are the same
          parent1, parent2 = random.choices(parents, k=2)
        print(color.On_IBlue+"random choices parent1 and parent2 for offspring:"+color.END, OFF_Count)
        OFF_Count +=1
        print("parent1", parent1)
        print("parent2", parent2)
        child = {}
        for param in search_space.keys():
            parent1_binary = bin(int(parent1[param]))[2:]  # Convert integer to binary
            parent2_binary = bin(int(parent2[param]))[2:]  # Convert integer to binary

            # Add necessary leading zeros if required
            parent1_binary = '0' * (len(bin(max(int(parent1[param]), int(parent2[param])))[2:]) - len(parent1_binary)) + parent1_binary
            parent2_binary = '0' * (len(bin(max(int(parent1[param]), int(parent2[param])))[2:]) - len(parent2_binary)) + parent2_binary

            child_binary = ''
            for i in range(len(parent1_binary)):
                if random.random() < 0.3:
                    child_binary += parent1_binary[i]
                else:
                    child_binary += parent2_binary[i]

            child[param] = int(child_binary, 2)  # Convert binary to integer
            if random.random() < mutation_rate:
                mutated_value = random.choice(search_space[param])
                child[param] = mutated_value
                print(color.RED +color.BOLD+f"Mutated {param} to {mutated_value}"+color.END)
            elif child[param] == 0:
                child[param] = parent2[param]  # Replace the value with the parent's value
        offspring.append(child)

    Of_n = 1
    print(color.On_Red+"----------------------------------------------------------------------------------------------------------------------"+color.END)
    for item_4 in offspring:
        print(color.BLUE +color.BOLD+f"Offsprings {Of_n}: {item_4}"+color.END)
        Of_n += 1

    return offspring

# Evaluate the offspring architectures

In [8]:
def evaluate_offspring(offspring):
    print(color.On_Red+"----------------------------------------------------------------------------------------------------------------------"+color.END)
    Ch = 1
    for child in offspring:
        child['fitness'] = evaluate_architecture(child)
        print(color.GREEN +color.BOLD+f"child fitness {Ch}:"+color.END, child['fitness'])
        print(color.GREEN+"-------------------------------------------------------"+color.END)
        Ch +=1

# Combine parents, offspring, and top performers

In [9]:
def combine_population(offspring):
    population = offspring #parents + top performers
    for item_3 in population:
      print(item_3)
    return population

# Define the termination criterion

In [10]:
def termination_criterion(generation, max_generations):
    return generation >= max_generations

# NAS with genetic algorithm

In [11]:
def nas_ga(population_size, num_parents, num_offspring, num_generations, mutation_rate):
    population = generate_population(population_size)
    NN = 1
    for individual in population:
        individual['fitness'] = evaluate_architecture(individual)
        print(color.GREEN +color.BOLD+"individual:"+color.END, NN ,color.DARKCYAN +color.BOLD+"-------->> individual fitness:"+color.END, individual['fitness'])
        NN += 1
    for generation in range(num_generations):
        print(color.On_Purple+"********************************generation:"+color.END, generation+1, color.On_Purple+"**********************************************"+color.END)
        parents = select_parents(population, num_parents)
        offspring = generate_offspring(parents, num_offspring, mutation_rate)
        evaluate_offspring(offspring)
        population = combine_population(offspring)
        if termination_criterion(generation, num_generations):
            break

    best_architecture = max(population, key=lambda x: x['fitness'])
    print("Best architecture:", best_architecture)
    print(color.On_Purple+"evaluate test datse with Best architecture:"+color.END, evaluate_architecture(best_architecture, x_test, y_test))
    return best_architecture

# Run NAS with genetic algorithm

In [16]:
population_size = 10
num_parents = 5
num_generations = 10
mutation_rate = 0.2
best_architecture = nas_ga(population_size=population_size, num_parents=num_parents,
                           num_offspring=population_size, num_generations=num_generations, mutation_rate = mutation_rate)

Initial population of neural architectures--------------------------------------------------------------------------
Population: 1 {'num_conv_layers': 3, 'num_filters': 15, 'num_dense_units': 578, 'kernel_size': 4, 'pool_size': 3}
Population: 2 {'num_conv_layers': 6, 'num_filters': 2, 'num_dense_units': 915, 'kernel_size': 2, 'pool_size': 3}
Population: 3 {'num_conv_layers': 1, 'num_filters': 15, 'num_dense_units': 830, 'kernel_size': 5, 'pool_size': 4}
Population: 4 {'num_conv_layers': 1, 'num_filters': 10, 'num_dense_units': 822, 'kernel_size': 5, 'pool_size': 3}
Population: 5 {'num_conv_layers': 3, 'num_filters': 2, 'num_dense_units': 774, 'kernel_size': 3, 'pool_size': 2}
Population: 6 {'num_conv_layers': 1, 'num_filters': 15, 'num_dense_units': 823, 'kernel_size': 5, 'pool_size': 2}
Population: 7 {'num_conv_layers': 3, 'num_filters': 9, 'num_dense_units': 514, 'kernel_size': 2, 'pool_size': 4}
Population: 8 {'num_conv_layers': 6, 'num_filters': 4, 'num_dense_units': 714, 'kernel_s